In [ ]:
%matplotlib inline
import pandas as pd
import os
# Import the main functionality from the SimFin Python API.
import simfin as sf

# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

import yfinance as yf
import yahoo_fin.stock_info as si
# import pandas_datareader.data as web

from dateutil.relativedelta import relativedelta
from datetime import datetime

In [ ]:
# sample base and starting date for training
tickers_sp500 = list(si.tickers_sp500())
start_date = '2016-01-01'
end_date = '2021-12-31'

In [ ]:
cwd = os.getcwd()
cwd

In [ ]:
sf.set_data_dir('/Users/feiyiyang/Documents/Bootcamp/Team9-Project2/Bootcamp-Project2/simfin_data/')

In [ ]:
sf.load_api_key(path='/Users/feiyiyang/Documents/Bootcamp/Team9-Project2/Bootcamp-Project2/simfin_data/simfin_api_key.txt')

### Part 1: Grab quarterly fundamentals from Simfin API

In [ ]:
df_income = sf.load(dataset='income', variant='quarterly', market='us')
df_balance = sf.load(dataset='balance', variant='quarterly', market='us')
df_cashflow = sf.load(dataset='cashflow', variant='quarterly', market='us')

In [ ]:
df_cashflow.columns

In [ ]:
df_fundamentals = pd.merge(df_income[['Ticker','Fiscal Year','Fiscal Period','Report Date', 'Publish Date','Revenue','Gross Profit','Operating Income (Loss)','Net Income']], 
                           df_balance[['Ticker','Fiscal Year','Fiscal Period','Shares (Basic)','Cash, Cash Equivalents & Short Term Investments','Total Assets','Total Liabilities','Total Equity']],  
                           how='left', 
                           on=['Ticker','Fiscal Year','Fiscal Period'])

In [ ]:
df_fundamentals= pd.merge(df_fundamentals,
                          df_cashflow[['Ticker','Fiscal Year','Fiscal Period','Net Cash from Operating Activities','Net Change in Long Term Investment','Net Cash from Investing Activities', 'Dividends Paid', 'Net Cash from Financing Activities', 'Net Change in Cash']],
                           how='left', 
                           on=['Ticker','Fiscal Year','Fiscal Period'])

In [ ]:
df_fundamentals.tail()

In [ ]:
# add a column for next earning date - to set the start/end date for append daily price
df_fundamentals['Next_publish_date']=df_fundamentals.groupby("Ticker")['Publish Date'].shift(-1)
u = (pd.to_datetime(df_fundamentals['Publish Date']) + pd.DateOffset(months=3)).dt.date
df_fundamentals.Next_publish_date.fillna(u,inplace=True)

In [ ]:
df_fundamentals.to_csv('data/us_fundamental.csv',index=False)

### Part 2: Grab EARNING DATA from yahoo finance API (yahoo_fin.stock_info)

In [ ]:
# load fundamental data
df_fundamentals=pd.read_csv('data/us_fundamental.csv')

In [ ]:


# dftest=pd.DataFrame.from_dict(si.get_earnings_history("A"))[['ticker','startdatetime','epsestimate','epsactual','epssurprisepct']]
# dftest.startdatetime=pd.to_datetime(dftest.startdatetime).dt.date
# dftest=dftest[dftest['startdatetime']>pd.to_datetime(start_date).date()]
# dftest['earning_y']= pd.DatetimeIndex(dftest['startdatetime']).year
# dftest['earning_m']= pd.DatetimeIndex(dftest['startdatetime']).month
# dftest.sort_values(by=['startdatetime'],inplace=True)
# dftest

In [ ]:
earningdata_1=[]
notmatch=[]
for ticker in tickers_sp500:
    try:
        # print(ticker)
        dftest=pd.DataFrame.from_dict(si.get_earnings_history(ticker))[['ticker','startdatetime','epsestimate','epsactual','epssurprisepct']]
        dftest.startdatetime=pd.to_datetime(dftest.startdatetime).dt.date
        dftest=dftest[dftest['startdatetime']>pd.to_datetime(start_date).date()]
        dftest['earning_y']= pd.DatetimeIndex(dftest['startdatetime']).year
        dftest['earning_m']= pd.DatetimeIndex(dftest['startdatetime']).month
        dftest.sort_values(by=['startdatetime'],inplace=True)

        df1=df_fundamentals[df_fundamentals['Ticker']==ticker][['Ticker','Publish Date']]
        df1['Publish_y']=pd.DatetimeIndex(df1['Publish Date']).year
        df1['Publish_m']=pd.DatetimeIndex(df1['Publish Date']).month
    except:
        print(f"No data available for {ticker}")
        notmatch.append(ticker)
    else:
        merge= pd.merge(df1,dftest,how='left',left_on=('Ticker','Publish_y'),right_on=('ticker','earning_y'))
        merge= merge[(merge.earning_m == merge.Publish_m) | (merge.earning_m == merge.Publish_m-1)]
        earningdata_1.append(merge)
    

In [ ]:
notmatch3=[]
for ticker in ['FOX', 'NWS']:
    try:
        dftest=pd.DataFrame.from_dict(si.get_earnings_history(ticker))[['ticker','startdatetime','epsestimate','epsactual','epssurprisepct']]
        dftest.startdatetime=pd.to_datetime(dftest.startdatetime).dt.date
        dftest=dftest[dftest['startdatetime']>pd.to_datetime(start_date).date()]
        dftest['earning_y']= pd.DatetimeIndex(dftest['startdatetime']).year
        dftest['earning_m']= pd.DatetimeIndex(dftest['startdatetime']).month
        dftest.sort_values(by=['startdatetime'],inplace=True)

        df1=df_fundamentals[df_fundamentals['Ticker']==ticker][['Ticker','Publish Date']]
        df1['Publish_y']=pd.DatetimeIndex(df1['Publish Date']).year
        df1['Publish_m']=pd.DatetimeIndex(df1['Publish Date']).month
    except:
        print(f"No data available for {ticker}")
        notmatch3.append(ticker)
    else:
        merge= pd.merge(df1,dftest,how='left',left_on=('Ticker','Publish_y'),right_on=('ticker','earning_y'))
        merge= merge[(merge.earning_m == merge.Publish_m) | (merge.earning_m == merge.Publish_m-1)]
        earningdata_1.append(merge)

In [31]:
df_sp500_earning = pd.concat(earningdata_1)

In [32]:
df_sp500_earning=df_sp500_earning.reset_index()
df_sp500_earning.drop(columns=['index','earning_y','earning_m','Publish_y','Publish_m','ticker'],inplace=True)
df_sp500_earning.rename(columns={'startdatetime':'Earning Date'},inplace=True)
df_sp500_earning

,Ticker,Publish Date,Earning Date,epsestimate,epsactual,epssurprisepct
0,A,2016-09-07,2016-08-17,0.47,0.49,4.70
1,A,2016-12-20,2016-11-15,0.52,0.59,13.24
2,A,2017-03-08,2017-02-14,0.49,0.53,8.16
3,A,2017-06-06,2017-05-22,0.48,0.58,19.83
4,A,2017-09-06,2017-08-15,0.52,0.59,13.03
...,...,...,...,...,...,...
7884,WYNN,2020-05-08,2020-05-06,-0.84,-3.54,-321.43
7885,WYNN,2020-08-06,2020-08-04,-4.98,-6.14,-23.29
7886,WYNN,2020-11-09,2020-11-05,-4.03,-7.04,-74.91
7887,WYNN,2021-02-26,2021-02-04,-2.31,-2.45,-6.11


In [33]:
df_sp500_earning.to_csv('data/sp500_earningdate.csv',index=False)

### Part 3: Grab daily stock price from yahoo finance API (yahoo_fin.stock_info)

In [40]:
# load fundamental data
df_fundamentals=pd.read_csv('data/us_fundamental.csv')
df_sp500_earning=pd.read_csv('data/sp500_earningdate.csv')

In [34]:
# append daily price data to fundamental where the dACC_NOTES_RECVate is within the current publish date and next publish date
price=[]
for ticker in tickers_sp500:
    try:
        # df1=df_fundamentals[df_fundamentals['Ticker']==ticker]
        df2=si.get_data(ticker , start_date = start_date, end_date=end_date,index_as_date=False)
    except:
        print(f"No data available for {ticker}")
    else:
        # df_merge=pd.merge(df1,df2,how='left',left_on='Ticker',right_on='ticker')
        # df_merge= df_merge[(df_merge.date > df_merge['Publish Date']) & (df_merge.date <= df_merge['Next_publish_date'])]
        price.append(df2)
sp500_price = pd.concat(price)
sp500_price.to_csv('data/sp500_dailyprice.csv',index=False)

In [ ]:
# this is for testing if the merge was correct
#list(df_sp500[(df_sp500['Publish Date']== '2021-05-06') & (df_sp500['Ticker']== 'AES')].date)

,date,open,high,low,close,adjclose,volume,ticker
0,2016-01-04,46.144001,46.276001,43.799999,44.681999,44.681999,34135500,TSLA
1,2016-01-05,45.271999,45.377998,44.000000,44.686001,44.686001,15934000,TSLA
2,2016-01-06,44.000000,44.009998,43.195999,43.807999,43.807999,18895500,TSLA
3,2016-01-07,42.838001,43.688000,42.734001,43.130001,43.130001,17771500,TSLA
4,2016-01-08,43.571999,44.088001,42.153999,42.200001,42.200001,18140500,TSLA
...,...,...,...,...,...,...,...,...
1505,2021-12-23,1006.799988,1072.979980,997.559998,1067.000000,1067.000000,30904400,TSLA
1506,2021-12-27,1073.670044,1117.000000,1070.719971,1093.939941,1093.939941,23715300,TSLA
1507,2021-12-28,1109.489990,1119.000000,1078.420044,1088.469971,1088.469971,20108000,TSLA
1508,2021-12-29,1098.640015,1104.000000,1064.140015,1086.189941,1086.189941,18718000,TSLA


### Part 4: Append additional data source

#### 1.Industry/Sector

In [ ]:
# load fundamental data
df_sp500=pd.read_csv('data/sp500_fundamental_dailyprice.csv')

In [ ]:
df_industry_sector=pd.DataFrame(list(df_sp500.Ticker.unique()),columns=['Ticker'])

In [ ]:
test=df_industry_sector.iloc[:5]

In [ ]:
df2 = {'Ticker': 'KK'}
test = test.append(df2, ignore_index = True)
test

In [ ]:
sector_df= pd.DataFrame(columns=['Ticker', 'Sector', 'Industry'])

for ticker in df_industry_sector.Ticker.unique():
    try:
        sector_df=sector_df.append({
            'Ticker':ticker,
            'Sector':si.get_company_info(ticker).loc['sector'].Value,
            'Industry':si.get_company_info(ticker).loc['industry'].Value
        }, ignore_index=True)
        # print(ticker)
    except:
        print(f"No industry data available for {ticker}")


In [ ]:
sector_df.tail()

In [ ]:
sector_df.to_csv('data/industry_sector.csv',index=False)

In [ ]:
df_sp500=pd.merge(df_sp500,sector_df,how='left', on='Ticker')

In [ ]:
len(df_industry_sector.Ticker.unique())

#### 2.Earning Surprise

In [ ]:
dftest=pd.DataFrame.from_dict(si.get_earnings_history("TSLA"))[['ticker','startdatetime','epsestimate','epsactual','epssurprisepct']]
dftest.startdatetime=pd.to_datetime(dftest.startdatetime).dt.date
dftest=dftest[dftest['startdatetime']>pd.to_datetime("2016-1-1").date()]
dftest['earningestimate_year']= pd.DatetimeIndex(dftest['startdatetime']).year
dftest['earningestimate_month']= pd.DatetimeIndex(dftest['startdatetime']).month
dftest.sort_values(by=['startdatetime'],inplace=True)

In [ ]:
dftest['epsestimate_nextday'] = (pd.to_datetime(dftest['startdatetime']) + pd.DateOffset(days=1)).dt.date

In [ ]:
dftest

In [ ]:
df_sp500[df_sp500['Ticker']=='TSLA']['Publish Date'].unique()

In [ ]:
### Part 4: Target Data 

In [ ]:
# tickers_sp500
frames=[]
for ticker in ['AAPL','TSLA']:
    try:
        df1=df_fundamentals[df_fundamentals['Ticker']==ticker]['']
        df2=si.get_data(ticker , start_date = start_date, end_date=end_date,index_as_date=False)
    except:
        print(f"No data available for {ticker}")
    else:
        df_merge=pd.merge(df1,df2,how='left',left_on='Ticker',right_on='ticker')
        df_merge= df_merge[(df_merge.date > df_merge['Publish Date']) & (df_merge.date <= df_merge['Next_publish_date'])]
        frames.append(df_merge)
    

df_sp500 = pd.concat(frames)

In [ ]:
test=df_sp500[df_sp500['Ticker']=='TSLA'][['Ticker','Publish Date','date','close']]

In [ ]:
test['close_minus1']=test['close'].shift(1)
test['close_plus1']=test['close'].shift(-4)
test

In [ ]:
test['T0'] = (pd.to_datetime(test['Publish Date']) + pd.DateOffset(days=-1)).dt.date

In [ ]:
df_sp500[df_sp500['Ticker']=='A']